In [1]:
import nevergrad as ng
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

### Problem Formulation

The financial portfolio optimization problem is a classic challenge in finance that involves determining the optimal allocation of assets within an investment portfolio. The primary goal is to construct a portfolio that achieves the best possible trade-off between expected returns and risk, typically measured as volatility. Investors seek to maximize returns while minimizing the overall portfolio risk.

\begin{align*}
\text{Maximize: } & \sum_{i=1}^{N} w_i \cdot \text{expected\_returns}_i \\
\text{Minimize: } & \sqrt{\sum_{i=1}^{N} \sum_{j=1}^{N} w_i \cdot w_j \cdot \text{covariance\_matrix}_{ij}}
\end{align*}

### Sample Data

In [2]:
# Sample data
expected_returns_simulated = np.array([0.08, 0.12, 0.10, 0.15, 0.09])
covariance_matrix_simulated = np.array([[0.0004, 0.0003, 0.0002, 0.0001, 0.0002],
                                        [0.0003, 0.0009, 0.0004, 0.0002, 0.0003],
                                        [0.0002, 0.0004, 0.0010, 0.0003, 0.0004],
                                        [0.0001, 0.0002, 0.0003, 0.0008, 0.0002],
                                        [0.0002, 0.0003, 0.0004, 0.0002, 0.0005]])

### Solving the Bi-objective Problem Using Nevergrad

In [3]:
# Bi-objective problem: Financial portfolio optimization
def portfolio_optimization(weights, expected_returns, covariance_matrix):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights, np.dot(covariance_matrix, weights)))
    # maximize portfolio return and minimize portfolio volatility
    return [-portfolio_return, portfolio_volatility]

In [4]:
# Using Nevergrad to solve the bi-objective problem
optimizer = ng.optimizers.CMA(parametrization= 5, budget=100)

initial_weights = np.random.dirichlet(np.ones(5), size=1).flatten()  # Random initial weights

optimizer.tell(ng.p.Array(init=initial_weights.tolist()), [0, 1],
               portfolio_optimization(initial_weights, expected_returns_simulated, covariance_matrix_simulated))

# Minimize the bi-objective problem
optimizer.minimize(lambda x: portfolio_optimization(x, expected_returns_simulated, covariance_matrix_simulated), verbosity=0, constraint_violation=[lambda x: -np.sum(x) + 1, lambda x: np.min(x) - 0])

print("Pareto front:")
for param in sorted(optimizer.pareto_front(), key=lambda p: p.losses[0]):
    print(f"{param} with losses {param.losses}")

Pareto front:
Array{(5,)}:[ 4.07422718  3.18749452 -0.8086181   6.52846041 -0.61177429] with losses [-1.55178508  0.24948115]
Array{(5,)}:[ 5.59888016  2.65475213  0.15265018  5.99176153 -1.60148189] with losses [-1.53637655  0.24888212]
Array{(5,)}:[ 4.4370526   2.92985837  1.40351938  5.42276182 -1.92466819] with losses [-1.48709329  0.24316077]
Array{(5,)}:[ 2.73355545  4.31876944  0.07096375  5.90908784 -2.30481584] with losses [-1.42296289  0.23957396]
Array{(5,)}:[ 5.58726189  0.83394838  1.19572064  5.32402362 -0.81103786] with losses [-1.39223696  0.22502671]
Array{(5,)}:[ 2.4765618   3.07874651 -1.03635855  6.20338962 -0.49669123] with losses [-1.3497449   0.22042531]
Array{(5,)}:[ 4.477896    2.10264227  0.57044211  4.69324024 -0.99862112] with losses [-1.2817031   0.20604177]
Array{(5,)}:[ 4.95930599  2.19930198 -1.193006    5.28327932 -1.19138741] with losses [-1.22662715  0.20377239]
Array{(5,)}:[ 3.1448869   1.68483975 -0.35840263  6.20651668 -1.56733078] with losses [-1.

### Solving the Bi-objective Problem Using Raytune and Nevergrad

In [5]:
import time
import ray
import nevergrad as ng
from ray import train, tune
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.nevergrad import NevergradSearch

In [6]:
def evaluate(weight_1, weight_2, weight_3, weight_4, weight_5, alpha1, alpha2):
    weights = [weight_1, weight_2, weight_3, weight_4, weight_5]

    portfolio_return = sum(weights[i] * expected_returns_simulated[i] for i in range(5))

    portfolio_volatility = sum(
        weights[i] * weights[j] * covariance_matrix_simulated[i, j] 
        for i in range(5) for j in range(5)
    )

    portfolio_volatility = np.sqrt(portfolio_volatility)
    
    return -alpha1 * portfolio_return + alpha2 * portfolio_volatility

def objective(config):
    alpha1 = config["alpha1"]
    alpha2 = 1 - alpha1
    config["alpha2"] = alpha2

    weights = [config["weight_1"], config["weight_2"], config["weight_3"], config["weight_4"], config["weight_5"]]
    total_weight = sum(weights)
    normalized_weights = [w / total_weight for w in weights]
    config['weight_1'] = normalized_weights[0]
    config['weight_2'] = normalized_weights[1]
    config['weight_3'] = normalized_weights[2]
    config['weight_4'] = normalized_weights[3]
    config['weight_5'] = normalized_weights[4]

    score = evaluate(config["weight_1"], config["weight_2"], config["weight_3"], config["weight_4"], config["weight_5"], alpha1, alpha2)
    train.report({"mean_loss": score})

algo = NevergradSearch(
    optimizer=ng.optimizers.CMA,
)

algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)

num_samples = 100

search_config = {
    "weight_1": tune.uniform(0, 1),
    "weight_2": tune.uniform(0, 1),
    "weight_3": tune.uniform(0, 1),
    "weight_4": tune.uniform(0, 1),
    "weight_5": tune.uniform(0, 1),
    "alpha1": tune.uniform(0, 1),
    "alpha2": 0
}

tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

best_result_config = results.get_best_result().config
formatted_config = {key: round(value, 2) for key, value in best_result_config.items()}
print("Best results found were: ", formatted_config)

(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.p

Best results found were:  {'weight_1': 0.16, 'weight_2': 0.2, 'weight_3': 0.16, 'weight_4': 0.3, 'weight_5': 0.19, 'alpha1': 0.98, 'alpha2': 0.02}


(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.p

(raylet) /Users/peterkeszthelyi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
